In [ ]:
import numpy as np


In [ ]:
datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

In [ ]:
print(datain)
print(datain.shape)   # Useful if it’s an array
print(type(datain)) 

In [ ]:
print(dataout)
print(dataout.shape)   # Useful if it’s an array
print(type(dataout)) 

# Week 1

In [ ]:
from scipy.optimize import minimize
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

X_init = datain
y_init = dataout

# Fit GP
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_init)

# Surrogate to maximise (negative for minimize)
def surrogate_neg(x):
    return -gp.predict(x.reshape(1, -1))[0]

# Bounds for normalized inputs
bounds = [(0,1), (0,1), (0,1), (0,1)]

# Try multiple random starts to avoid local issues
best_x = None
best_val = float('inf')
for _ in range(10):
    x0 = np.random.rand(4)
    res = minimize(surrogate_neg, x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_val:
        best_val = res.fun
        best_x = res.x

x_next = best_x
print("Next point to evaluate:", x_next)


In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

# --- Existing data ---
X_init = datain           # shape (n_samples, 4)
y_init = dataout          # shape (n_samples,)

# --- New evaluated point ---
x_new = np.array([0.973386, 0.889905, 0.981563, 0.242055])
y_new = 2755.4496930419423

# --- Add new data to dataset ---
X_all = np.vstack([X_init, x_new])
y_all = np.hstack([y_init, y_new])

# --- Refit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_all, y_all)

# --- Generate candidate next points around current best ---
best_x = x_new
sigma = 0.02  # tweak size for local exploration
num_candidates = 5

x_next_candidates = best_x + np.random.normal(0, sigma, size=(num_candidates, 4))
# Ensure all points are within [0,1]
x_next_candidates = np.clip(x_next_candidates, 0, 1)

print("Candidate next points to evaluate:")
print(x_next_candidates)
